In [44]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [47]:
class Stock:

    def __init__(self, path, name):
        self.name = name
        self.path = path
        self.dfs = []
        self.quarters = []
        self.master_df = None
        self.driver = None

    def import_data_tables(self):
        """
        Retrieves the Income statement, Balance Sheet and Cash Flow for each of the last five quarters
        From the webpage's HTML 
        Combines each statement of a single quarter into a dataFrame and stores it the the instance's df list
        """
        def import_statements(*data):
            # create lists representing each column for all three tables combined
            # Metric 
            list1 = []
            # Value (K, M, B in USD, or No units)
            list2 = []
            # Y/Y Change (%)
            list3 = []
            # add the columns' data to the appropriate list
            for dt in data:
                for d in dt:
                    if dt == metric:
                        list1.append(d.get_attribute('innerHTML'))
                    elif dt == values:
                        val = d.get_attribute('innerHTML')
                        # Determine the units and convert to proper dollar amount as float type
                        if val[-1] == "B":
                            val = float(val[0:-1]) * 1000000000
                        elif val[-1] == "M":
                            val = float(val[0:-1]) * 1000000
                        elif val[-1] == "K":
                            val = float(val[0:-1]) * 1000
                        else:
                            val = float(val[0:-1])
                        list2.append(val)
                    elif dt == change:
                        chg = d.get_attribute('innerHTML')
                        chg = str(chg)[-18:].strip("</span>")
                        # Remove the % sign to be able to convert to float
                        if chg[-1] == "%":
                            chg = float(''.join(chg[0:-1].split(',')))
                        # If there is no % sign, then the change is not tracked and should be replaced with None
                        else:
                            chg = None
                        list3.append(chg)
            # zip into an array-like structure and convert to DataFrame
            list4 = zip(list1, list2, list3)
            df = pd.DataFrame(data=list4, columns=['Metric', 'Value', 'Y/Y Change'])
            return df

        # column headers
        header = self.driver.find_elements(by=By.CLASS_NAME, value='yNnsfe')
        # Store the financial statement quarter Date (Month/Year) in the quarters list
        self.quarters.append(header[1].get_attribute('innerHTML')[:8])
        # value column
        values = self.driver.find_elements(by=By.CLASS_NAME, value='QXDnM')
        # change column
        change = self.driver.find_elements(by=By.CLASS_NAME, value="gEUVJe")
        # metric column
        metric = self.driver.find_elements(by=By.CLASS_NAME, value='rsPbEe')
        # Create a dataFrame for the quarter and store it in the dfs list
        self.dfs.append(import_statements(metric, values, change))

        print("------\nData tables imported successfully\n------")

    def create_master_df(self):
        # Combine each quarter's financial statemnt into a master dataFrame
        if self.dfs:
            self.master_df = pd.concat(self.dfs,
                                       keys=[AMD.quarters[0], AMD.quarters[1], AMD.quarters[2], AMD.quarters[3], AMD.quarters[4]],
                                       names=["Quarter", 'Row ID'])

    def start_driver(self):
        # Initialize the driver
        chrome_options = Options()
        chrome_options.add_experimental_option('detach', True)
        self.driver = webdriver.Chrome(options=chrome_options)
        # Open the target Indices webpage on Google Finance
        self.driver.get(f"https://www.google.com/finance/quote/{self.path}")
        # Open the Balance Sheet and Cash Flow tabs (income statement tab is opened upon loading the webpage)
        self.driver.execute_script("document.getElementsByClassName('oX8Xbb Tj1T2')[1].click()")
        self.driver.execute_script("document.getElementsByClassName('oX8Xbb Tj1T2')[2].click()")

    def close_driver(self):
        # Close the driver's connection 
        if self.driver:
            self.driver.close()


# Instantiate an instance of the Stock class 
AMD = Stock('AMD:NASDAQ', 'AMD')

In [48]:
try:
    AMD.start_driver()
    # cycle through each quarterly statement and inmport the tabular data
    for i in range(1, 6):
        elem = AMD.driver.find_element(by=By.XPATH, value='//*[@id="yDmH0d"]/c-wiz[2]/div/div[4]/div/main/div[2]/c-wiz/div/div[2]/div/div[1]/div')
        AMD.driver.execute_script('arguments[0].scrollIntoView()', elem)
        if i != 1:
            buttons = AMD.driver.find_elements(by=By.XPATH, value=f'//*[@id="option-{i - 1}"]')
            for btn in buttons:
                btn.click()
        AMD.import_data_tables()
finally:
    AMD.close_driver()

------
Data tables imported successfully
------
------
Data tables imported successfully
------
------
Data tables imported successfully
------
------
Data tables imported successfully
------
------
Data tables imported successfully
------


In [49]:
AMD.create_master_df()
# copy the master dataFrame for later manipulations without compromising the original data
df = AMD.master_df.copy()

In [50]:
# Get the size of the dataset
df.shape

(105, 3)

In [51]:
# Get the names and data types of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 105 entries, ('Mar 2024', 0) to ('Apr 2023', 20)
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Metric      105 non-null    object 
 1   Value       105 non-null    float64
 2   Y/Y Change  75 non-null     float64
dtypes: float64(2), object(1)
memory usage: 3.1+ KB


In [52]:
# Get the number of unique values for each categorical variable
unique_values = df.select_dtypes(include=['object', 'category']).nunique()
unique_values

Metric    20
dtype: int64

In [54]:
# Get the min and max values for each column containing numerical data
max_val, min_val = df['Value'].max(), df['Value'].min()
max_change, min_change = df['Y/Y Change'].max(), df['Y/Y Change'].min()
max_val, min_val, max_change, min_change

(67970000000.0, -1240000000.0, 3076.19, -145.8)

In [42]:
# Determine number of empty column values
df.isna().sum()

Metric         0
Value          0
Y/Y Change    30
dtype: int64

In [55]:
# Drop the associated rows containing the empty values
df.dropna()

Metric         Value  Y/Y Change
Quarter  Row ID                                                
Mar 2024 0                    Revenue  5.470000e+09        2.24
         1          Operating expense  2.750000e+09       -1.96
         2                 Net income  1.230000e+08      188.49
         3          Net profit margin  2.200000e+00      186.54
         4         Earnings per share  6.000000e-01        3.33
...                               ...           ...         ...
Apr 2023 16      Cash from operations  4.860000e+08      -51.16
         17       Cash from investing -1.240000e+09     -139.17
         18       Cash from financing -2.590000e+08       86.70
         19        Net change in cash -1.010000e+09     -145.80
         20            Free cash flow  7.037500e+08      -14.31

[75 rows x 3 columns]

In [57]:
# get the number of instances of each categorial value (Metric column)
df['Metric'].value_counts()

Metric
Net income                         10
Revenue                             5
Shares outstanding                  5
Net change in cash                  5
Cash from financing                 5
Cash from investing                 5
Cash from operations                5
Return on capital                   5
Return on assets                    5
Price to book                       5
Total equity                        5
Operating expense                   5
Total liabilities                   5
Total assets                        5
Cash and short-term investments     5
Effective tax rate                  5
EBITDA                              5
Earnings per share                  5
Net profit margin                   5
Free cash flow                      5
Name: count, dtype: int64

In [ ]:
# The dataset conatins AMD's Financial statements from the past five quarters
# Each financial statement contains the Income statement, Balance Sheet, and Cash Flow
# Net Income is present in both the Income Statement and Cash Flow and therefore is most frequent through the dataset